# First Analysis: Linking Climate to Yield (1981 Test Case)

**Goal:** To perform a proof-of-concept analysis by linking our newly downloaded climate data with the corresponding crop yield data for the year 1981.

**Methodology:**
1.  Load the 1981 maize yield data.
2.  Load the January 1981 temperature data.
3.  Process the hourly temperature data into a single metric for the month (average temperature).
4.  Align the high-resolution climate grid to the lower-resolution yield grid.
5.  Create a scatter plot to visualize the relationship between temperature and yield for each grid cell.

In [2]:
# Cell 1: Load Data
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- Load Maize Yield Data for 1981 ---
YIELD_PATH = '../data/maize/yield_1981.nc4'
ds_yield = xr.open_dataset(YIELD_PATH)
print("Yield data for 1981 loaded.")

# --- Load Climate Data for Jan 1981 ---
CLIMATE_PATH = '../data/climate_raw/api_test/era5_land_usa_1981_01_temp_robust.grib'
# Use the cfgrib engine, which we know works
ds_climate = xr.open_dataset(CLIMATE_PATH, engine='cfgrib')
print("Climate data for Jan 1981 loaded.")

Yield data for 1981 loaded.
Climate data for Jan 1981 loaded.


C:\ProgramData\miniconda3\envs\climarisc\lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(


## Data Processing and Alignment

We need to make the two datasets compatible. This involves calculating a single temperature metric for the month and then resampling the climate grid to match the yield grid.

In [4]:
# Cell 2: Process and Align

# 1. Process Climate Data: Calculate the mean temperature for the month and convert to Celsius
# The climate data has a 'valid_time' dimension for the hours of the month. We average over it.
# We subtract 273.15 to convert from Kelvin to Celsius.
avg_temp_jan = ds_climate['t2m'].mean(dim='time') - 273.15

# 2. Align Grids: Resample the high-res temperature map to the low-res yield grid
print("Aligning climate grid to yield grid...")
avg_temp_aligned = avg_temp_jan.interp_like(ds_yield)

# 3. Combine into a single dataset
# We'll rename the yield variable for clarity
analysis_ds = xr.Dataset({
    'yield': ds_yield['var'],
    'avg_temp': avg_temp_aligned
})

print("\nData processed and aligned successfully.")
print(analysis_ds)

Aligning climate grid to yield grid...

Data processed and aligned successfully.
<xarray.Dataset> Size: 7MB
Dimensions:    (lon: 720, lat: 360, step: 24, latitude: 251, longitude: 251)
Coordinates:
  * lon        (lon) float64 6kB 0.25 0.75 1.25 1.75 ... 358.2 358.8 359.2 359.8
  * lat        (lat) float64 3kB -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
    number     int64 8B ...
  * step       (step) timedelta64[ns] 192B 01:00:00 02:00:00 ... 1 days 00:00:00
    surface    float64 8B ...
  * latitude   (latitude) float64 2kB 50.0 49.9 49.8 49.7 ... 25.2 25.1 25.0
  * longitude  (longitude) float64 2kB -105.0 -104.9 -104.8 ... -80.1 -80.0
Data variables:
    yield      (lat, lon) float32 1MB ...
    avg_temp   (step, latitude, longitude) float32 6MB -6.325 -6.462 ... nan nan


## First Scatter Plot: Yield vs. Temperature

Now we can plot the yield of each grid cell against the average January temperature of that same grid cell. This is the first step toward building a vulnerability curve.

In [5]:
# Cell 3: Create the Scatter Plot

# Convert the xarray Dataset to a pandas DataFrame, which is ideal for this kind of plotting.
# This turns our 2D maps into a list of paired (temperature, yield) values.
df = analysis_ds.to_dataframe()

# Drop any rows that have missing data (e.g., ocean grid cells)
df_clean = df.dropna()

print(f"Plotting {len(df_clean)} data points...")

# Create the scatter plot using seaborn for a nice look
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_clean, x='avg_temp', y='yield', alpha=0.5, s=10) # s=10 makes points smaller

plt.title('Maize Yield vs. Average January Temperature (US Midwest, 1981)')
plt.xlabel('Average January Temperature (°C)')
plt.ylabel('Maize Yield (tonnes per hectare)')
plt.grid(True)
plt.show()

MemoryError: Unable to allocate 1.43 TiB for an array with shape (720, 360, 24, 251, 251) and data type float32